In [109]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from torch.utils.data import DataLoader
from torch.optim import AdamW
from evaluate import load
from seq2seq import create_transformers_train_data, train_transformer, decode_with_transformer
from sklearn.metrics import accuracy_score

In [110]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [111]:
data = pd.read_csv('../yelp_parallel/yelp_parallel/test_en_parallel.txt', sep='\t')

In [112]:
negative = data["Style 1"].values.tolist()
positive = data["Style 2"].values.tolist()

In [113]:
print(positive[:10])
print(negative[:10])

["Ever since joes has changed hands it's gotten better and better.", 'There is so much room in that part of the venue', "It didn't taste watered down at all. ", "She said she'd be back, and didn't disappear at all. ", 'This pharmacy is really considerate.', 'just left and put it on the bill.', "it isn't perfect, but it is very good.", 'definitely not disappointed that i could use my birthday gift!', "great new owner, i heard - but i don't know the details.", "but it probably doesn't suck too!"]
["ever since joes has changed hands it's just gotten worse and worse.", 'there is definitely not enough room in that part of the venue.', 'so basically tasted watered down.', "she said she'd be back and disappeared for a few minutes.", "i can't believe how inconsiderate this pharmacy is.", 'just left and took it off the bill.', "it isn't terrible, but it isn't very good either.", 'definitely disappointed that i could not use my birthday gift!', "new owner, i heard - but i don't know the details.

In [114]:
bleu = load("bleu")
bertscore = load("bertscore")

In [146]:
def evaluation(model_name, negative, positive, lr=0.001, epochs=5, num_examples=30, batch_size=256, device=None):
    print(f"Model: {model_name}, Learning rate: {lr}, Epochs: {epochs}")

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

    train_dataset = create_transformers_train_data(negative, positive, tokenizer)
    data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, collate_fn=data_collator)

    optimizer = AdamW(model.parameters(), lr=lr)
    train_transformer(model, train_loader, optimizer, epochs, device=device)

    all_predictions = []
    all_references = []
    all_inputs = []

    for i in range(num_examples):
        input_text = negative[i]
        reference = positive[i]

        prediction = decode_with_transformer(input_text, tokenizer, model, device=device)

        all_inputs.append(input_text)
        all_predictions.append(prediction)
        all_references.append(reference)

        print("Negative: ", input_text)
        print("Prediction: ", prediction)
        print("Positive: ", reference)

    bleu_score = bleu.compute(
        predictions=all_predictions,
        references=all_references
    )

    bert_score = bertscore.compute(
        predictions=all_predictions,
        references=all_references,
        lang="en" #model_type='microsoft/deberta-xlarge-mnli' predolgo trae koga se koristi ovoj model_type pa zatoa koristam lang='en' bidejki samiot bert_score barashe barem edno od ovie da bide navedeno
    )

    avg_bert_f1 = sum(bert_score["f1"]) / len(bert_score["f1"])

    print("BLEU:", bleu_score)
    print("BERTScore f1: ", avg_bert_f1)
    # print("BERTScore: ", bert_score) premnogu golem output dava pa go skrativ za da bide popregledno

    return {
        "bleu": bleu_score,
        "BERTScore average f1": avg_bert_f1,
        # "bertscore": bert_score,
    }

In [147]:
test = evaluation("t5-small", negative, positive, lr=0.001, epochs=3, num_examples=30, device=device)

Model: t5-small, Learning rate: 0.001, Epochs: 3


C:\Users\arsan\PycharmProjects\opj\.venv\lib\site-packages\transformers\tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Epoch 1/3, Loss: 3.0152
Epoch 2/3, Loss: 2.3736
Epoch 3/3, Loss: 2.1823
Negative:  ever since joes has changed hands it's just gotten worse and worse.
Prediction:  ever since joes has changed hands
Positive:  Ever since joes has changed hands it's gotten better and better.
Negative:  there is definitely not enough room in that part of the venue.
Prediction:  there is definitely enough room in that part of
Positive:  There is so much room in that part of the venue
Negative:  so basically tasted watered down.
Prediction:  so basically tasted watered down.
Positive:  It didn't taste watered down at all. 
Negative:  she said she'd be back and disappeared for a few minutes.
Prediction:  she said she'd be back and disappeared
Positive:  She said she'd be back, and didn't disappear at all. 
Negative:  i can't believe how inconsiderate this pharmacy is.
Prediction:  i can't believe how inconsider
Positive:  This pharmacy is really considerate.
Negative:  just left and took it off the bill.
Pre

In [117]:
results = []

In [118]:
results.append(evaluation("t5-small", negative, positive, lr=0.001, epochs=3, num_examples=300, device=device))
results.append(evaluation("t5-small", negative, positive, lr=0.001, epochs=5, num_examples=300, device=device))
results.append(evaluation("t5-small", negative, positive, lr=0.0001, epochs=10, num_examples=300, device=device))

Model: t5-small, Learning rate: 0.001, Epochs: 3


C:\Users\arsan\PycharmProjects\opj\.venv\lib\site-packages\transformers\tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Epoch 1/3, Loss: 3.0156
Epoch 2/3, Loss: 2.3619
Epoch 3/3, Loss: 2.1632
Negative:  ever since joes has changed hands it's just gotten worse and worse.
Prediction:  ever since joes has changed hands
Positive:  Ever since joes has changed hands it's gotten better and better.
Negative:  there is definitely not enough room in that part of the venue.
Prediction:  there is definitely enough room in that part of
Positive:  There is so much room in that part of the venue
Negative:  so basically tasted watered down.
Prediction:  so much
Positive:  It didn't taste watered down at all. 
Negative:  she said she'd be back and disappeared for a few minutes.
Prediction:  she said she'd be back and disappeared
Positive:  She said she'd be back, and didn't disappear at all. 
Negative:  i can't believe how inconsiderate this pharmacy is.
Prediction:  i can't believe how inconsider
Positive:  This pharmacy is really considerate.
Negative:  just left and took it off the bill.
Prediction:  just left and to

In [119]:
results.append(evaluation("google/flan-t5-small", negative, positive, lr=0.001, epochs=3, num_examples=300, device=device))
results.append(evaluation("google/flan-t5-small", negative, positive, lr=0.001, epochs=5, num_examples=300, device=device))
results.append(evaluation("google/flan-t5-small", negative, positive, lr=0.0001, epochs=10, num_examples=300, device=device))

Model: google/flan-t5-small, Learning rate: 0.001, Epochs: 3


C:\Users\arsan\PycharmProjects\opj\.venv\lib\site-packages\transformers\tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Epoch 1/3, Loss: 2.6785
Epoch 2/3, Loss: 2.1359
Epoch 3/3, Loss: 1.9184
Negative:  ever since joes has changed hands it's just gotten worse and worse.
Prediction:  ever since joes has changed hands
Positive:  Ever since joes has changed hands it's gotten better and better.
Negative:  there is definitely not enough room in that part of the venue.
Prediction:  there is definitely enough room in that part of
Positive:  There is so much room in that part of the venue
Negative:  so basically tasted watered down.
Prediction:  so basically tasted watered down.
Positive:  It didn't taste watered down at all. 
Negative:  she said she'd be back and disappeared for a few minutes.
Prediction:  she said she'd be back and gone
Positive:  She said she'd be back, and didn't disappear at all. 
Negative:  i can't believe how inconsiderate this pharmacy is.
Prediction:  i can't believe how inconsider
Positive:  This pharmacy is really considerate.
Negative:  just left and took it off the bill.
Prediction

In [120]:
for i in range(len(results)):
    print(f"{i}: {results[i]}")

0: {'bleu': {'bleu': 0.24762017824426225, 'precisions': [0.5635514018691589, 0.39021739130434785, 0.2811688311688312, 0.20225624496373892], 'brevity_penalty': 0.7404721248129001, 'length_ratio': 0.7689543657923105, 'translation_length': 2140, 'reference_length': 2783}, 'BERTScore average f1': 0.9126658109823863}
1: {'bleu': {'bleu': 0.2601733967215699, 'precisions': [0.583216127519925, 0.41025641025641024, 0.29810828440965426, 0.21735604217356042], 'brevity_penalty': 0.7373186532806957, 'length_ratio': 0.7664390945023356, 'translation_length': 2133, 'reference_length': 2783}, 'BERTScore average f1': 0.915349870522817}
2: {'bleu': {'bleu': 0.2316726938585628, 'precisions': [0.5488437942425672, 0.37053326003298515, 0.26201448321263987, 0.18934426229508197], 'brevity_penalty': 0.7309901044704289, 'length_ratio': 0.7614085519223859, 'translation_length': 2119, 'reference_length': 2783}, 'BERTScore average f1': 0.9095540358622869}
3: {'bleu': {'bleu': 0.2687422521052555, 'precisions': [0.57

TASK 2

In [121]:
instruction_prompt = "Translate this negative sentence into a positive one: "

In [122]:
instructional_negative = [instruction_prompt + sentence for sentence in negative]

In [123]:
test = evaluation("t5-small", instructional_negative, positive, lr=0.0001, epochs=5, num_examples=10, device=device)

Model: t5-small, Learning rate: 0.0001, Epochs: 5


C:\Users\arsan\PycharmProjects\opj\.venv\lib\site-packages\transformers\tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Epoch 1/5, Loss: 6.3306
Epoch 2/5, Loss: 5.2375
Epoch 3/5, Loss: 4.8250
Epoch 4/5, Loss: 4.5877
Epoch 5/5, Loss: 4.4148
Negative:  Translate this negative sentence into a positive one: ever since joes has changed hands it's just gotten worse and worse.
Prediction:  this is a positive one: since 
Positive:  Ever since joes has changed hands it's gotten better and better.
Negative:  Translate this negative sentence into a positive one: there is definitely not enough room in that part of the venue.
Prediction:  i have a great time.
Positive:  There is so much room in that part of the venue
Negative:  Translate this negative sentence into a positive one: so basically tasted watered down.
Prediction:  i have a very good time.
Positive:  It didn't taste watered down at all. 
Negative:  Translate this negative sentence into a positive one: she said she'd be back and disappeared for a few minutes.
Prediction:  ''I'm not a
Positive:  She said she'd be back, and didn't disappear at all. 
Negativ

In [124]:
test2 = evaluation("t5-small", instructional_negative, positive, lr=0.0001, epochs=5, num_examples=10, device=device)

Model: t5-small, Learning rate: 0.0001, Epochs: 5


C:\Users\arsan\PycharmProjects\opj\.venv\lib\site-packages\transformers\tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Epoch 1/5, Loss: 6.4093
Epoch 2/5, Loss: 5.2735
Epoch 3/5, Loss: 4.8710
Epoch 4/5, Loss: 4.6330
Epoch 5/5, Loss: 4.4400
Negative:  Translate this negative sentence into a positive one: ever since joes has changed hands it's just gotten worse and worse.
Prediction:  joes has changed hands and has
Positive:  Ever since joes has changed hands it's gotten better and better.
Negative:  Translate this negative sentence into a positive one: there is definitely not enough room in that part of the venue.
Prediction:  Cette sentence est d'une bonne
Positive:  There is so much room in that part of the venue
Negative:  Translate this negative sentence into a positive one: so basically tasted watered down.
Prediction:      
Positive:  It didn't taste watered down at all. 
Negative:  Translate this negative sentence into a positive one: she said she'd be back and disappeared for a few minutes.
Prediction:  ''I'm not a
Positive:  She said she'd be back, and didn't disappear at all. 
Negative:  Transl

In [125]:
test3 = evaluation("t5-small", instructional_negative, positive, lr=0.0001, epochs=10, num_examples=10, device=device)

Model: t5-small, Learning rate: 0.0001, Epochs: 10


C:\Users\arsan\PycharmProjects\opj\.venv\lib\site-packages\transformers\tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Epoch 1/10, Loss: 6.5539
Epoch 2/10, Loss: 5.3983
Epoch 3/10, Loss: 4.9682
Epoch 4/10, Loss: 4.6816
Epoch 5/10, Loss: 4.4625
Epoch 6/10, Loss: 4.3363
Epoch 7/10, Loss: 4.2211
Epoch 8/10, Loss: 4.1507
Epoch 9/10, Loss: 4.1021
Epoch 10/10, Loss: 4.0436
Negative:  Translate this negative sentence into a positive one: ever since joes has changed hands it's just gotten worse and worse.
Prediction:  the food was great and the food was great
Positive:  Ever since joes has changed hands it's gotten better and better.
Negative:  Translate this negative sentence into a positive one: there is definitely not enough room in that part of the venue.
Prediction:  the food was excellent and the food was excellent
Positive:  There is so much room in that part of the venue
Negative:  Translate this negative sentence into a positive one: so basically tasted watered down.
Prediction:  the food was great and the food was great
Positive:  It didn't taste watered down at all. 
Negative:  Translate this negati

Ne razbiram zoshto celo vreme vika deka "the food was great" izgleda epten mu se dopadnala. Ama sepak probuvav dosta za da go promenam modelot za da mozhe da proraboti koristejkji razlichni prompts no ednostavno ne se promeni vo nekoj sluchaevi duri i na germanski mi vrati vo eden od testovite pogore koga gi gledav rechenicite shto gi predviduva

In [126]:
test4 = evaluation("google/flan-t5-small", instructional_negative, positive, lr=0.0001, epochs=10, num_examples=10, device=device)

Model: google/flan-t5-small, Learning rate: 0.0001, Epochs: 10


C:\Users\arsan\PycharmProjects\opj\.venv\lib\site-packages\transformers\tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Epoch 1/10, Loss: 4.4250
Epoch 2/10, Loss: 3.9742
Epoch 3/10, Loss: 3.7803
Epoch 4/10, Loss: 3.6785
Epoch 5/10, Loss: 3.6021
Epoch 6/10, Loss: 3.5590
Epoch 7/10, Loss: 3.5055
Epoch 8/10, Loss: 3.4611
Epoch 9/10, Loss: 3.4268
Epoch 10/10, Loss: 3.3934
Negative:  Translate this negative sentence into a positive one: ever since joes has changed hands it's just gotten worse and worse.
Prediction:  i'm not a fan of
Positive:  Ever since joes has changed hands it's gotten better and better.
Negative:  Translate this negative sentence into a positive one: there is definitely not enough room in that part of the venue.
Prediction:  the food was great and the service was great
Positive:  There is so much room in that part of the venue
Negative:  Translate this negative sentence into a positive one: so basically tasted watered down.
Prediction:  i'm a fan of the
Positive:  It didn't taste watered down at all. 
Negative:  Translate this negative sentence into a positive one: she said she'd be back

In [127]:
test5 = evaluation("google/flan-t5-small", instructional_negative, positive, lr=0.0001, epochs=5, num_examples=10, device=device)

Model: google/flan-t5-small, Learning rate: 0.0001, Epochs: 5


C:\Users\arsan\PycharmProjects\opj\.venv\lib\site-packages\transformers\tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Epoch 1/5, Loss: 4.4142
Epoch 2/5, Loss: 3.9882
Epoch 3/5, Loss: 3.7636
Epoch 4/5, Loss: 3.6732
Epoch 5/5, Loss: 3.5983
Negative:  Translate this negative sentence into a positive one: ever since joes has changed hands it's just gotten worse and worse.
Prediction:  i love the sashimi
Positive:  Ever since joes has changed hands it's gotten better and better.
Negative:  Translate this negative sentence into a positive one: there is definitely not enough room in that part of the venue.
Prediction:  i love the food and service.
Positive:  There is so much room in that part of the venue
Negative:  Translate this negative sentence into a positive one: so basically tasted watered down.
Prediction:  i love the steamed rice and
Positive:  It didn't taste watered down at all. 
Negative:  Translate this negative sentence into a positive one: she said she'd be back and disappeared for a few minutes.
Prediction:  i was so excited to see her again
Positive:  She said she'd be back, and didn't disap

BLEU rezultatite se generalno za nikade.

TASK 3

In [128]:
instruction_prompt

'Translate this negative sentence into a positive one: '

In [129]:
instructional_negative[:10]

["Translate this negative sentence into a positive one: ever since joes has changed hands it's just gotten worse and worse.",
 'Translate this negative sentence into a positive one: there is definitely not enough room in that part of the venue.',
 'Translate this negative sentence into a positive one: so basically tasted watered down.',
 "Translate this negative sentence into a positive one: she said she'd be back and disappeared for a few minutes.",
 "Translate this negative sentence into a positive one: i can't believe how inconsiderate this pharmacy is.",
 'Translate this negative sentence into a positive one: just left and took it off the bill.',
 "Translate this negative sentence into a positive one: it isn't terrible, but it isn't very good either.",
 'Translate this negative sentence into a positive one: definitely disappointed that i could not use my birthday gift!',
 "Translate this negative sentence into a positive one: new owner, i heard - but i don't know the details.",
 'T

In [130]:
classification_prompt = "Classify the following text into negative or positive using one word: "

In [131]:
classification_negative = [classification_prompt + sentence for sentence in negative]
classification_positive = [classification_prompt + sentence for sentence in positive]
print(classification_negative[:3])
print(classification_positive[:3])

["Classify the following text into negative or positive using one word: ever since joes has changed hands it's just gotten worse and worse.", 'Classify the following text into negative or positive using one word: there is definitely not enough room in that part of the venue.', 'Classify the following text into negative or positive using one word: so basically tasted watered down.']
["Classify the following text into negative or positive using one word: Ever since joes has changed hands it's gotten better and better.", 'Classify the following text into negative or positive using one word: There is so much room in that part of the venue', "Classify the following text into negative or positive using one word: It didn't taste watered down at all. "]


In [132]:
negative_labels = ["negative"] * len(negative)
positive_labels = ["positive"] * len(positive)
print(negative_labels[:3])
print(positive_labels[:3])

['negative', 'negative', 'negative']
['positive', 'positive', 'positive']


In [136]:
classif_inputs = classification_negative + classification_positive
classif_outputs = negative_labels + positive_labels

In [137]:
print(instructional_negative[:10])
print(positive[:10])

["Translate this negative sentence into a positive one: ever since joes has changed hands it's just gotten worse and worse.", 'Translate this negative sentence into a positive one: there is definitely not enough room in that part of the venue.', 'Translate this negative sentence into a positive one: so basically tasted watered down.', "Translate this negative sentence into a positive one: she said she'd be back and disappeared for a few minutes.", "Translate this negative sentence into a positive one: i can't believe how inconsiderate this pharmacy is.", 'Translate this negative sentence into a positive one: just left and took it off the bill.', "Translate this negative sentence into a positive one: it isn't terrible, but it isn't very good either.", 'Translate this negative sentence into a positive one: definitely disappointed that i could not use my birthday gift!', "Translate this negative sentence into a positive one: new owner, i heard - but i don't know the details.", 'Translate 

In [138]:
inputs = instructional_negative + classif_inputs
outputs = positive + classif_outputs

In [141]:
print(inputs[:10])
print(outputs[:10])
print(inputs[-15:-10])
print(outputs[-15:-10])

["Translate this negative sentence into a positive one: ever since joes has changed hands it's just gotten worse and worse.", 'Translate this negative sentence into a positive one: there is definitely not enough room in that part of the venue.', 'Translate this negative sentence into a positive one: so basically tasted watered down.', "Translate this negative sentence into a positive one: she said she'd be back and disappeared for a few minutes.", "Translate this negative sentence into a positive one: i can't believe how inconsiderate this pharmacy is.", 'Translate this negative sentence into a positive one: just left and took it off the bill.', "Translate this negative sentence into a positive one: it isn't terrible, but it isn't very good either.", 'Translate this negative sentence into a positive one: definitely disappointed that i could not use my birthday gift!', "Translate this negative sentence into a positive one: new owner, i heard - but i don't know the details.", 'Translate 

In [142]:
print(len(inputs))
print(len(outputs))

3000
3000


In [148]:
input_trans = instructional_negative.copy()
ref_trans = positive.copy()
input_class = classif_inputs.copy()
ref_class = classif_outputs.copy()


In [149]:
print(input_trans[:10])
print(ref_trans[:10])
print(input_class[:10])
print(ref_class[:10])

["Translate this negative sentence into a positive one: ever since joes has changed hands it's just gotten worse and worse.", 'Translate this negative sentence into a positive one: there is definitely not enough room in that part of the venue.', 'Translate this negative sentence into a positive one: so basically tasted watered down.', "Translate this negative sentence into a positive one: she said she'd be back and disappeared for a few minutes.", "Translate this negative sentence into a positive one: i can't believe how inconsiderate this pharmacy is.", 'Translate this negative sentence into a positive one: just left and took it off the bill.', "Translate this negative sentence into a positive one: it isn't terrible, but it isn't very good either.", 'Translate this negative sentence into a positive one: definitely disappointed that i could not use my birthday gift!', "Translate this negative sentence into a positive one: new owner, i heard - but i don't know the details.", 'Translate 

In [150]:
print(len(input_trans))
print(len(ref_trans))
print(len(input_class))
print(len(ref_class))

1000
1000
2000
2000


In [154]:
def evaluateT3(model_name, inputs, outputs, lr=0.001, epochs=5, num_examples=50, batch_size=128, device=device):
    print("Task: 3")
    print(f"Model: {model_name}, Learning rate: {lr}, Epochs: {epochs}")

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

    train_dataset  = create_transformers_train_data(inputs, outputs, tokenizer)

    data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, collate_fn=data_collator)

    optimizer = AdamW(model.parameters(), lr=lr)

    train_transformer(model, train_loader, optimizer, epochs, device=device)

    all_predictions = []
    all_references = []
    all_inputs = []

    for i in range(1000):
        input_text = input_trans[i]
        reference = ref_trans[i]

        prediction = decode_with_transformer(input_text, tokenizer, model, device=device)

        all_inputs.append(input_text)
        all_predictions.append(prediction)
        all_references.append(reference)

        # print("Input: ", input_text)
        # print("Prediction: ", prediction)
        # print("Positive: ", reference)

    bleu_score = bleu.compute(
        predictions=all_predictions,
        references=all_references
    )

    bert_score = bertscore.compute(
        predictions=all_predictions,
        references=all_references,
        lang="en" #model_type='microsoft/deberta-xlarge-mnli' predolgo trae koga se koristi ovoj model_type pa zatoa koristam lang='en' bidejki samiot bert_score barashe barem edno od ovie da bide navedeno
    )

    avg_bert_f1 = sum(bert_score["f1"]) / len(bert_score["f1"])

    print("BLEU:", bleu_score)
    print("BERTScore f1: ", avg_bert_f1)


    predicted_labels = []

    for i in range(2000):
        input_text = input_class[i]
        reference = ref_class[i]
        pred = decode_with_transformer(input_text, tokenizer, model, device=device).lower().strip()

        if reference in pred:
            predicted_label = reference
        else:
            predicted_label = pred

        predicted_labels.append(predicted_label)

    correct = sum(p == r for p, r in zip(predicted_labels, ref_class))
    accuracy = correct / len(ref_class) if len(ref_class) > 0 else 0
    print("Accuracy:", accuracy)

    return {
        "bleu": bleu_score,
        "bert_f1": avg_bert_f1,
        "classification_accuracy": accuracy
    }


In [155]:
results_t3 = evaluateT3(
    model_name="t5-small",
    inputs=inputs,
    outputs=outputs,
    lr=0.0005,
    epochs=5,
    num_examples=50,
    batch_size=64,
    device=device
)

print(results_t3)

Task: 3
Model: t5-small, Learning rate: 0.0005, Epochs: 5


C:\Users\arsan\PycharmProjects\opj\.venv\lib\site-packages\transformers\tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Epoch 1/5, Loss: 2.0317
Epoch 2/5, Loss: 2.0501
Epoch 3/5, Loss: 1.9289
Epoch 4/5, Loss: 1.6762
Epoch 5/5, Loss: 1.4991
BLEU: {'bleu': 0.01319671416613402, 'precisions': [0.1522491349480969, 0.02186204297022239, 0.006610145135795373, 0.0018459473065950663], 'brevity_penalty': 0.9296015563577887, 'length_ratio': 0.9319671278477062, 'translation_length': 8959, 'reference_length': 9613}
BERTScore f1:  0.8751716538667679
Accuracy: 0.5
{'bleu': {'bleu': 0.01319671416613402, 'precisions': [0.1522491349480969, 0.02186204297022239, 0.006610145135795373, 0.0018459473065950663], 'brevity_penalty': 0.9296015563577887, 'length_ratio': 0.9319671278477062, 'translation_length': 8959, 'reference_length': 9613}, 'bert_f1': 0.8751716538667679, 'classification_accuracy': 0.5}
